In [1]:
import boto3
import os

In [5]:
access_key_id = "minioadmin"
secret_access_key = "minioadmin"
minio_url = "http://localhost:9000"

minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

new_bucket = "formatted-zone"

try:
    minio_client.head_bucket(Bucket=new_bucket)
except:
    minio_client.create_bucket(Bucket=new_bucket)

In [ ]:
import io
import re
import unicodedata
from tqdm import tqdm

bucket_origen = "persistent-landing"
bucket_desti = "formatted-zone"
prefix_origen = "textos/"

# Llistar tots els .txt del bucket d'origen , ho fem de forma automatitzada per si en un futur
# hem de carregar més textos
paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_origen, Prefix=prefix_origen):
    for obj in tqdm(page.get("Contents", []), desc="Processant fitxers de text"):
        key = obj["Key"]

        # Nom del fitxer (sense path)
        filename = key.split("/")[-1]

        #Llegir el contingut del fitxer original
        response = minio_client.get_object(Bucket=bucket_origen, Key=key)
        text = response["Body"].read().decode("utf-8", errors="ignore")

        # APLICAR TRATAMIENTOS COMPLETOS DE LIMPIEZA Y MEJORA DEL TEXTO
        
        # 1. Normalización Unicode (unificar caracteres especiales)
        text_clean = unicodedata.normalize('NFKD', text)
        
        # 2. Eliminar caracteres de control y no imprimibles
        text_clean = ''.join(char for char in text_clean if unicodedata.category(char)[0] != 'C' or char in '\n\t')
        
        # 3. Limpiar espacios en blanco múltiples
        text_clean = re.sub(r"[ \t]+", " ", text_clean)          # Espacios y tabs múltiples → un espacio
        
        # 4. Limpiar saltos de línea múltiples
        text_clean = re.sub(r"\n\s*\n\s*\n+", "\n\n", text_clean) # Múltiples líneas vacías → máximo 2
        
        # 5. Eliminar espacios al inicio y final de líneas
        lines = text_clean.split('\n')
        lines = [line.strip() for line in lines]
        text_clean = '\n'.join(lines)
        
        # 6. Eliminar líneas completamente vacías al inicio y final
        lines = [line for line in lines if line.strip()]  # Eliminar líneas vacías del medio también
        
        # 7. Limpiar caracteres especiales problemáticos
        text_clean = re.sub(r'[^\w\s\.\,\;\:\!\?\(\)\[\]\"\'\-\+\=\%\&\$\/\@\#\*]', '', text_clean)
        
        # 8. Normalizar comillas y apostrofes
        text_clean = re.sub(r'["""]', '"', text_clean)      # Comillas tipográficas → comillas normales
        text_clean = re.sub(r'[''']', "'", text_clean)      # Apostrofes tipográficos → apostrofes normales
        
        # 9. Limpiar guiones múltiples
        text_clean = re.sub(r'-{2,}', '-', text_clean)      # Múltiples guiones → un guión
        
        # 10. Normalizar espacios alrededor de puntuación
        text_clean = re.sub(r'\s+([.!?;:])', r'\1', text_clean)  # Eliminar espacios antes de puntuación
        text_clean = re.sub(r'([.!?;:])\s*', r'\1 ', text_clean) # Un espacio después de puntuación
        
        # 11. Limpiar espacios al inicio y final del texto completo
        text_clean = text_clean.strip()
        
        # 12. Asegurar que el texto no esté vacío
        if not text_clean.strip():
            print(f"Advertencia: {filename} quedó vacío después del procesamiento")
            continue
        
        # Convertir a bytes i pujar-lo al bucket formatted-zone
        new_key = f"textos/{filename}"

        minio_client.put_object(
            Bucket=bucket_desti,
            Key=new_key,
            Body=text_clean.encode("utf-8")
        )

Processant fitxers de text: 100%|██████████| 1/1 [00:00<00:00,  5.95it/s]
